# CoVid-19 Tracing

Use this in case it doesn't connect

In [1]:
#! mkdir data

In [2]:
#cd ./data
#%cd ./data/COVID-19

In [3]:
#!git clone https://github.com/CSSEGISandData/COVID-19.git
#!git pull

In [4]:
#%cd ../..

##### We are going to use the data from CSSEGISandData 

In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
from geopy.extra.rate_limiter import RateLimiter
from datetime import date, datetime, timedelta
import bokeh
import pandas_bokeh
from pandas_profiling import ProfileReport

### 1. Data acquisition

We must get the files from the source:

In [ ]:
files=[]
# The path where the files are is:#
path = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/'
#path = './data/COVID-19/csse_covid_19_data/csse_covid_19_daily_reports/'
# We must get the name of each file. All the are dates, so:

start = start_date=date(2020, 1, 22)
end = date.today()
date_generated = [start + timedelta(days=x) for x in range(0, (end-start).days)]

for dates in date_generated:
    dates=dates.strftime("%m-%d-%Y")+'.csv'
    files.append(dates)

def dataframe(files):
    try:
        datos=pd.DataFrame()
        for file in files:
            filepath=path+file
            df_file=pd.read_csv(filepath)
            datos=pd.concat([datos,df_file],axis=0)
        return datos
    except TimeoutError:
        return dataframe(files)
datos=dataframe(files)

In [7]:
df=datos.copy()

### Cleaning data

Data must be cleaned to get a better result. It should be checked periodically.

In [8]:
df.rename(columns={'Country/Region':'Country'},inplace=True)
df.rename(columns={'Province/State':'Province'},inplace=True)
# Some names must be cleaned manually
df.loc[df['Province']=='Fench Guiana','Province']='French Guiana'
df.loc[df['Province']=='Hong Kong','Country'] = 'Hong Kong'
df.loc[df['Province']=='Macau','Country'] = 'Macao'
df.loc[df['Province']=='Taiwan','Country'] = 'Taiwan'
df.loc[df['Province']=='UK','Province'] = np.NaN
df.loc[df['Country'].str.startswith('Tai'),'Country'] = 'Taiwan'
#Some of them are twice but with different names
list_unique_s=list(df['Province'].unique())
list_unique_s.remove(np.NaN)
for state in list_unique_s:
    filtro=df['Province'].str.contains(state).fillna(False)
    unicos=df.loc[filtro,'Province'].unique()
    if len(unicos)>1:
        df.loc[filtro,'Province']=unicos[0]

list_unique_c=list(df['Country'].unique())
for country in list_unique_c:
    filtro=df['Country'].str.contains(country).fillna(False)
    unicos=df.loc[filtro,'Country'].unique()
    if len(unicos)>1:
        df.loc[filtro,'Country']=unicos[0]
        
# In some rows the Country is placed in the 'Province/State' column
for i in range(df.shape[0]):
    if df.iloc[i,0]==df.iloc[i,1]:
        df.iloc[i,0]=np.NaN
    else:
        continue

/home/joeldela/anaconda3/lib/python3.7/site-packages/pandas/core/strings.py:1843: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


#### Data Localization

In [9]:
df_aux=df.copy()
df_aux=df_aux.reset_index().drop('index',axis=1)

We may get the Address based on the coordimates:

In [85]:
df_aux['Address']=np.NaN

The loop to use is:

```python
geolocator = Nominatim(user_agent="CoVid-19",timeout=10)
geocode = RateLimiter(geolocator.reverse, min_delay_seconds=1)
for i in range(df_aux.shape[0]):
    try:
        if not pd.isnull(df_aux['Address'][i]):
            df_aux['Address'][i]
        elif pd.isnull(df_aux['Latitude'][i]):
            df_aux['Address'][i]=np.NaN
        else:
            lat=df_aux['Latitude'][i]
            lon=df_aux['Longitude'][i]
            df_aux['Address'][i]=str(geocode(str(lat)+', '+str(lon)))
    except TypeError:
        continue
```

In [ ]:
def locate(row):
    geolocator = Nominatim(user_agent="CoVid-19")
    geocode = RateLimiter(geolocator.reverse, min_delay_seconds=1)
    try:
        if not pd.isnull(df_aux['Address'][i]):
            df_aux['Address'][i]
        elif pd.isnull(df_aux['Latitude'][i]):
            df_aux['Address'][i]=np.NaN
        else:
            lat=df_aux['Latitude'][i]
            lon=df_aux['Longitude'][i]
            df_aux['Address'][i]=str(geocode(str(lat)+', '+str(lon)))
    except TypeError:
        continue

df_aux.apply(lambda row: locate(row),axis=1)

In [113]:
df_aux['Location']=np.nan
for i in range(df.shape[0]):
    try:
        if pd.isnull(df_aux['Address'][i]):
            df_aux['Address'][i]
        elif any(i.isdigit() for i in df_aux['Address'][i].split(',')[-2]):
            df_aux['Location'][i]=df_aux['Address'][i].split(',')[-3]
        else:
            df_aux['Location'][i]=df_aux['Address'][i].split(',')[-2]
    except IndexError:
        df_aux['Address'][i]

/home/joeldela/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/home/joeldela/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [114]:
df_aux

,Confirmed,Country,Deaths,Last Update,Latitude,Longitude,Province,Recovered,Address,Location
0,1.0,Mainland China,NaN,1/22/2020 17:00,NaN,NaN,Anhui,NaN,NaN,NaN
1,14.0,Mainland China,NaN,1/22/2020 17:00,NaN,NaN,Beijing,NaN,NaN,NaN
2,6.0,Mainland China,NaN,1/22/2020 17:00,NaN,NaN,Chongqing,NaN,NaN,NaN
3,1.0,Mainland China,NaN,1/22/2020 17:00,NaN,NaN,Fujian,NaN,NaN,NaN
4,NaN,Mainland China,NaN,1/22/2020 17:00,NaN,NaN,Gansu,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
6717,0.0,Guernsey,0.0,2020-03-17T18:33:03,49.4500,-2.580,NaN,0.0,"Route des Talbots, Saint Peter Port, Guernsey,...",Guernsey
6718,0.0,Jersey,0.0,2020-03-17T18:33:03,49.1900,-2.110,NaN,0.0,"6, Rouge Bouillon, St Helier, JE2 3PP, Jersey",St Helier
6719,0.0,Puerto Rico,0.0,2020-03-17T16:13:14,18.2000,-66.500,NaN,0.0,"Cacos, Orocovis, Puerto Rico, United States of...",Puerto Rico
6720,0.0,Republic of the Congo,0.0,2020-03-17T21:33:03,-1.4400,15.556,NaN,0.0,"Abala, Plateaux, République du Congo",Plateaux


In [122]:
data_covid=df_aux[['Province','Country','Confirmed','Deaths','Last Update','Latitude','Latitude','Longitude','Address','Location']]
data_covid.loc[:,'Last Update']=pd.to_datetime(data_covid.loc[:,'Last Update'])

##### Now we can proceed with the data visualization